In [32]:
import psycopg2

import pandas as pd
import numpy as np

from copy import deepcopy

import ast

import networkx as nx

import itertools

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from joblib import Parallel, delayed

In [2]:
connection = psycopg2.connect('dbname=link_formation host=localhost user=postgres password=1_sehir_1')

twitter_users = pd.read_sql("SELECT * FROM twitter_user", connection)

user_connections = pd.read_sql("SELECT * FROM twitter_connection", connection).drop('id', axis=1)

In [4]:
twitter_users.sample(5)

,id,name,screen_name,lang,match_name,match_ratio,followers_count,friends_count
19646,535425870,Aykut Kurt,AykutKurt13,tr,Aykut Ucar,67,97,122
8905,706425190,Büsra,Bsrayrkcglu,tr,Busra Yorukcuoglu,79,261,290
12447,1007383164,osman urper,osmanurper,tr,Osman Nurettin Sen,72,260,249
22147,2962557225,Abdullah Demir,a_demirr55,tr,Kadem,72,139,273
2584,926331128100937728,Murat Demirci,MuratDe49231258,tr,Murat Er,68,2,100


In [5]:
twitter_users = pd.read_sql("SELECT * FROM twitter_user", connection)
twitter_users = twitter_users.where(twitter_users.match_name.str.len()>6)\
                             .where(~twitter_users.name.str.contains("(?i)sehir"))\
                             .dropna().set_index("id")
#         .where(twitter_users.match_ratio>85)
twitter_users.sample(5)

,name,screen_name,lang,match_name,match_ratio,followers_count,friends_count
id,,,,,,,
3.955128e+09,esra,esrakpk1,tr,Esra Kucuk,67.0,5.0,131.0
1.858418e+09,Burak,BurakErenTR,tr,Burak Eren,86.0,308.0,256.0
1.498686e+09,Gevrek Gündem,canozcan22,tr,Selin Banu Ozcan,63.0,466.0,3580.0
3.972518e+08,z e y n e p,zeynpchn,tr,Zeynep Cihan,80.0,141.0,223.0
9.289916e+17,Samet Ay,SametAy93763453,tr,! Some,68.0,0.0,94.0


In [6]:
truncate = lambda x: int(str(int(x))[:9])

In [7]:
resetted = twitter_users.reset_index()
index = resetted.id.apply(truncate)
resetted.rename(columns={"id":"tw_id"}, inplace=True)
resetted.index = index

twitter_users = resetted
twitter_users.sample(5)

,tw_id,name,screen_name,lang,match_name,match_ratio,followers_count,friends_count
id,,,,,,,,
504378378,5.043784e+08,Mustafa Zengin,mustafazengins,tr,Mustafa Talip Zengin,76.0,218.0,237.0
378779702,3.787797e+08,Rukiye YOLDAŞ,rukiyeyolds,tr,Rukiye Aydin,70.0,50.0,62.0
720976032,7.209760e+08,Bekir G. Çekirge,bahadirbektas61,tr,Ahmet Bahadir Bektas,74.0,139.0,176.0
427666672,4.276667e+08,Melis,degisiikinsann,tr,! Isif,68.0,614.0,1002.0
159950817,1.599508e+09,7den70eNano,7den70eNano,tr,! Dekan,54.0,41.0,98.0


In [43]:
import time

In [63]:
def clean(name, min_len=5, junk_replacement=''):
    try:
        cleaned = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').lower().decode("ascii")
    except TypeError:
        return junk_replacement
    if len(cleaned) < min_len:
        return junk_replacement
    return cleaned

def get_matches_edit_distance(item, choices, limit, scorer=fuzz.token_sort_ratio):
    return process.extract(item, choices, limit=limit, scorer=scorer)
counter = 0
def get_sehir_twitter_matches(twitter_users, sehir_directory, limit=1):
    global fullnames, counter
    twitter_user_by_screen_name = twitter_users.set_index('screen_name')
    start = time.time()
    for screen_name in twitter_users['screen_name']:
        twitter_name = twitter_user_by_screen_name.loc[screen_name]['name']
        match_name = get_matches_edit_distance(twitter_name, fullnames, limit)
        counter += 1
        if counter %100 == 0:
            print(counter, "out of ", len(twitter_users))
            start_ = time.time()
            print(start_-start, "seconds")
            start = start_
        yield (screen_name, match_name)
        
def filter_matches_by_threshold(matches_dict, threshold=85):
    filtered_dict = dict()
    for screen_name, matches in matches_dict.items():
        filtered = [(match, score) for match, score in matches if score > threshold]
        
        if filtered:
            filtered_dict[screen_name] = filtered
        
    return filtered_dict

def get_matches_dataframe(twitter_users, sehir_directory, threshold=85, limit=1):
    matches = {screen_name : match_name for screen_name, match_name in 
               get_sehir_twitter_matches(twitter_users, sehir_directory, limit=limit)}
    
    filtered_matches = filter_matches_by_threshold(matches, threshold=threshold)
    screen_names = filtered_matches.keys()
    return pd.DataFrame({'screen_name': list(screen_names),
                         'match_name': [filtered_matches[screen_name] for screen_name in screen_names]})

In [64]:
sehir_directory = pd.read_csv('../datasets/contacts.csv', 
                               encoding = "ISO-8859-1", 
                               usecols=['First Name', 'Last Name', 'Primary Email'])
sehir_directory.replace(np.nan, '', regex=True, inplace=True)

In [65]:
fullnames = [' '.join(first_last_name).lower() 
                 for first_last_name in sehir_directory[['First Name', 'Last Name']].values]

In [66]:
start = time.time()
sehir_matches = Parallel(n_jobs=-1)(delayed(get_matches_dataframe)(
    twitter_users[int(i*(len(twitter_users)/8)):int((i+1)*(len(twitter_users)/8))],
    sehir_directory) for i in range(8))
print("took: ", time.time()-start)

100 out of  2607
17.601778984069824 seconds
100 out of  2607
17.979655981063843 seconds
100 out of  2607
18.0801682472229 seconds
100 out of  2607
18.159130811691284 seconds
100 out of  2607
18.274653673171997 seconds
100 out of  2607
18.388170957565308 seconds
100 out of  2607


19.683095455169678 seconds
100 out of  2606
20.11982250213623 seconds


200 out of  2607
18.50369381904602 seconds
200 out of  2607
18.751721382141113 seconds
200 out of  2607
18.925511598587036 seconds
200 out of  2607
19.635064601898193 seconds
200 out of  2607
19.997876405715942 seconds
200 out of  2607
19.642518043518066 seconds
200 out of  2607
18.974292993545532 seconds
200 out of  2606
18.664579153060913 seconds


300 out of  2607
18.837618589401245 seconds
300 out of  2607
18.51528811454773 seconds
300 out of  2607
18.670929670333862 seconds
300 out of  2607
18.36874222755432 seconds
300 out of  2607
18.477618932724 seconds


300 out of  2606
18.260138988494873 seconds
300 out of  2607
19.428550004959106 seconds


...

2500 out of  2607
19.281773328781128 seconds
2500 out of  2607
18.590744018554688 seconds


2500 out of  2607
19.140007734298706 seconds


2600 out of  2606
19.329821586608887 seconds
2600 out of  2607
19.440301179885864 seconds


2600 out of  2607
18.718732595443726 seconds
2600 out of  2607
18.349258184432983 seconds


2600 out of  2607
19.233174324035645 seconds
2600 out of  2607
18.336395740509033 seconds
2600 out of  2607
17.82541561126709 seconds
2600 out of  2607
16.10215926170349 seconds
took:  491.66715931892395


In [69]:
sehir_matches_df = pd.concat(sehir_matches)
sehir_matches_df.index = range(len(sehir_matches_df))
print("There are {} matches".format(len(sehir_matches_df)))
sehir_matches_df.sample(5)

There are 2089 matches


,match_name,screen_name
1117,"[(sinan yavuz, 100)]",Sinan53Yavuz
1196,"[(zeynep ulusan, 92)]",UlusalZ
784,"[(yasir bali, 90)]",A_AliJnr
754,"[(fatma seker, 86)]",FatimaFtmanur
1885,"[(gulcin kayhan, 96)]",GulinKayhan


In [70]:
sehir_matches_df.match_name = sehir_matches_df.match_name.apply(lambda x: x[0][0])
sehir_matches_df.sample(10)

,match_name,screen_name
660,omer can,mertcantwitt
542,esra balanli,balanli_esra
786,mehmet demir,mdemir_61
1628,aysel mutlu,Ayse__Mutlu
1502,! tarih,tarihdergi
109,omer faruk karakaya,KarakulaOmer
1315,seyma arslan,eseyman
1109,elyesa turgut,elyesaturgut
864,abdullah mursel,AbdullahMursel
1970,abdurrahman yavuz,ayvz25


In [87]:
tu=twitter_users.drop(["match_name","match_ratio"], axis=1)

In [110]:
twitter_users = sehir_matches_df.merge(tu, on="screen_name")
index = twitter_users.tw_id.apply(truncate)
twitter_users.index = index
twitter_users.sample(5)

,match_name,screen_name,tw_id,name,lang,followers_count,friends_count
tw_id,,,,,,,
291820987,serkan akcay,serkankocaa,2.918210e+09,Serkan KOCA,tr,1042.0,707.0
249663199,derya guven,deryaguvenn,2.496632e+09,Derya Güven,tr,88.0,125.0
147551716,samet sevgi,SametSvg,1.475517e+09,Samet Sevgi,en,13.0,72.0
306589073,ayhan yilmaz,TweetAyhan,3.065891e+09,Ayhan Yilmaz,en,35.0,289.0
233713046,merve kandemir,Mrve_Dmr1453,2.337130e+09,Merve Demir,tr,46.0,317.0


In [111]:
twitter_users.to_csv("../datasets/twitter_users.csv", index_label="id")

## Construct the network

In [112]:
G = nx.DiGraph()
for _, row in user_connections.iterrows():
    from_ = truncate(row["from_user_id"])
    to = truncate(row["to_user_id"])
    if from_ in twitter_users.index and to in twitter_users.index:
        G.add_edge(from_, to)

In [113]:
import unicodedata
augs = ["name", "screen_name","match_name", "followers_count","friends_count"]
for node in G.nodes():
    user = twitter_users.loc[node]
    for aug in augs:
        if type(user[aug])==str:
            m = clean(user[aug])
        else:
            m = user[aug]
        G.nodes[node][aug] = m

In [114]:
len(G.nodes())

238

In [115]:
len(G.edges())

291

In [116]:
for ix,deg in G.degree(G.nodes()):
    G.node[ix]['degree'] = deg
    G.node[ix]['parity'] = (1-deg%2)
    
for ix,in_deg in G.in_degree(G.nodes()):
    G.node[ix]['in_degree'] = in_deg
    
for ix,out_deg in G.out_degree(G.nodes()):
    G.node[ix]['out_degree'] = out_deg

In [117]:
evc = nx.eigenvector_centrality(G)
closeness = nx.closeness_centrality(G)
betweenness = nx.betweenness_centrality(G)
pagerank = nx.pagerank(G)

In [118]:
metrics = {"eigenvector_centrality":evc,
           "closeness_centrality":closeness,
          "betweenness":betweenness,
          "pagerank":pagerank}

In [125]:
for metric_name, metric in metrics.items():
    for ix,v in metric.items():
        G.nodes[ix][metric_name] = v

In [130]:
G.nodes[595297280]

{'betweenness': 0.0,
 'closeness_centrality': 0.0,
 'degree': 1,
 'eigenvector_centrality': 8.290679621515326e-35,
 'followers_count': 71.0,
 'friends_count': 310.0,
 'in_degree': 0,
 'match_name': 'ayhan turkoglu',
 'name': 'talha turkoglu',
 'out_degree': 1,
 'pagerank': 0.0008750669358888037,
 'parity': 0,
 'screen_name': 'talha_turkoglu'}

In [120]:
list(G.nodes(data=True))[0]

(595297280,
 {'betweenness': 0.0,
  'closeness_centrality': 0.0,
  'degree': 1,
  'eigenvector_centrality': 8.290679621515326e-35,
  'followers_count': 71.0,
  'friends_count': 310.0,
  'in_degree': 0,
  'match_name': 'ayhan turkoglu',
  'name': 'talha turkoglu',
  'out_degree': 1,
  'pagerank': 0.0008750669358888037,
  'parity': 0,
  'screen_name': 'talha_turkoglu'})

In [122]:
import json
from networkx.readwrite import json_graph
data = nx.node_link_data(G)
with open('../REST/static/networks/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)